In [9]:
import numpy as np 
import pandas as pd 
import os 
import datetime
from datetime import datetime
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, TimeoutError, Lock, Value, current_process, cpu_count
import data_preprocess_function as func
from functools import partial

In [10]:
#Read Index Data
filepath = "D:\\Strategic_Trading\\index\\price\\"
file_list = os.listdir(filepath)
df_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.read_and_join, file_list), total=len(file_list)), 1):
            df_list.append(x)
            
data = pd.concat(df_list)

In [11]:
data['this_volume'] = data['累積成交數量']-data['累積成交數量'].shift(periods=1,fill_value=0)
data.loc[(data['時間']=='09:00:00'),'this_volume'] = 0

In [12]:
df_list = [group[1] for group in data.groupby('日期')]
OHLCV_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.OHLCV_index, frequency=1), df_list), total=len(df_list)), 1):
                OHLCV_list.append(x)

df = pd.concat(OHLCV_list)

In [16]:
df = df.reset_index()
df.columns = ["ts", "open_index", "high_index", "low_index", "close_index", "vol_index"]
df = df.sort_values(by=['ts'])

In [25]:
df["return_index"] = df["close_index"] - df["close_index"].shift(periods=1)

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df

,ts,open_index,high_index,low_index,close_index,vol_index,return_index
2981,2013-07-01 09:00:00,8062.21,8062.21,8001.59,8001.59,63737,NaN
2982,2013-07-01 09:01:00,7999.10,7999.10,7986.50,7986.50,24203,-15.09
2983,2013-07-01 09:02:00,7977.51,7977.51,7974.79,7975.63,25091,-10.87
2984,2013-07-01 09:03:00,7974.19,7974.39,7974.15,7974.39,32681,-1.24
2985,2013-07-01 09:04:00,7975.86,7985.52,7975.86,7985.52,34843,11.13
...,...,...,...,...,...,...,...
404869,2019-07-29 13:26:00,10904.60,10904.60,10904.60,10904.60,0,0.00
404870,2019-07-29 13:27:00,10904.60,10904.60,10904.60,10904.60,0,0.00
404871,2019-07-29 13:28:00,10904.60,10904.60,10904.60,10904.60,0,0.00
404872,2019-07-29 13:29:00,10904.60,10904.60,10904.60,10904.60,0,0.00


In [ ]:
df.to_csv("D:\\Strategic_Trading\\index\\min_agg_all.csv", index=False)